# HW 4 - All About Attention

Welcome to CS 287 HW4. To begin this assignment first turn on the Python 3 and GPU backend for this Colab by clicking `Runtime > Change Runtime Type` above.  

In this homework you will be reproducing the decomposable attention model in Parikh et al. https://aclweb.org/anthology/D16-1244. (This is one of the models that inspired development of the transformer). 



## Goal

We ask that you finish the following goals in PyTorch:

1. Implement the vanilla decomposable attention model as described in that paper.
2. Implement the decomposable attention model with intra attention or another extension.
3. Visualize the attentions in the above two parts.
4. Implement a mixture of models with uniform prior and perform training with exact log marginal likelihood (see below for detailed instructions)
5. Train the mixture of models in part 4 with VAE. (This may not produce a better model, this is still a research area) 
6. Interpret which component specializes at which type of tasks using the posterior.

Consult the paper for model architecture and hyperparameters, but you are also allowed to tune the hyperparameters yourself. 

In [2]:
from load_data import load
from models import AttentionModel, LatentVariableMixtureModel

import torch
# Text text processing library and methods for pretrained word embeddings
import torchtext
from torchtext.vocab import Vectors, GloVe

# Named Tensor wrappers
from namedtensor import ntorch, NamedTensor
from namedtensor.text import NamedField
import random
import copy

In [204]:
import numpy as np

In [4]:
train_iter, val_iter, test_iter, TEXT, LABEL = load(batch_size=1)

In [495]:
import imp
import models
m = imp.reload(models)

In [528]:
enc = m.VariationalEncoder(TEXT, LABEL, 5, 25)

In [515]:
batch = next(iter(train_iter))
# model = m.AttentionModel(TEXT, LABEL, 100, 100, intra_attn = True)
# model(batch.premise, batch.hypothesis).shape

In [529]:
model = m.AttentionModel(TEXT, LABEL, 100, 100, intra_attn = True)
mixture_model = m.LatentVariableMixtureModel(model, 5, enc)

In [530]:
mixture_model.reinforce(batch.premise, batch.hypothesis, batch.label, 5)

-1.3995503187179565

In [532]:
opt = torch.optim.Adam(mixture_model.parameters(), lr=.01)

In [534]:
acc = []
for ibatch, batch in enumerate(val_iter):
    pred = mixture_model(batch.premise, batch.hypothesis).argmax('logprob')
    acc.append((pred == batch.label).item())
    if ibatch > 100:
        break
np.mean(acc)

0.08823529411764706

In [533]:
elbos = []
for ibatch, batch in enumerate(train_iter):
    opt.zero_grad()
    elbo = mixture_model.reinforce(batch.premise, batch.hypothesis, batch.label, sample_size=5)
    elbos.append(elbo)
    opt.step()
    if ibatch % 10 == 0 and ibatch > 0:
        print(np.mean(elbos[-10:]))

-1.692370069026947
-1.9319700121879577
-2.2197653532028196
-2.316955828666687
-2.4686511754989624
-2.9517684459686278
-3.0035041093826296
-3.004321074485779
-3.0031389474868773
-3.0039746522903443
-3.0007352113723753
-3.0017306804656982
-3.0073405742645263
-3.0036590337753295
-3.001726841926575


KeyboardInterrupt: 

### Instructions for latent variable mixture model.

For the last part of this assignment we will consider a latent variable version of this model. This is a use of latent variable as a form of ensembling.

Instead of a single model, we use $K$ models $p(y | \mathbf{a}, \mathbf{b}; \theta_k)$ ($k=1,\cdots,K$), where $K$ is a hyperparameter. Let's introduce a discrete latent variable $c\sim \text{Uniform}(1,\cdots, K)$ denoting which model is being used to produce the label $y$, then the marginal likelihood is


$$
p(y|\mathbf{a}, \mathbf{b}; \theta) = \sum_{c=1}^K p(c) p(y | \mathbf{a}, \mathbf{b}; \theta_c)
$$

When $K$ is small, we can *enumerate* all possible values of $c$ to maximize the log marginal likelihood. 

We can also use variational auto encoding to perform efficient training. We first introduce an inference network $q(c| y, \mathbf{a}, \mathbf{b})$, and the ELBO is

$$
\log p(y|\mathbf{a}, \mathbf{b}; \theta)  \ge \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \log p(y|\mathbf{a},\mathbf{b}; \theta_c) - KL(q(c|y, \mathbf{a}, \mathbf{b})|| p(c)),
$$

where $p(c)$ is the prior uniform distribution. We can calculate the $KL$ term in closed form, but for the first term in ELBO, due to the discreteness of $c$, we cannot use the reparameterization trick. Instead we use REINFORCE to estimate the gradients (or see slides):

$$
\nabla \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \log p(y|\mathbf{a},\mathbf{b}; \theta_c) = \mathbb{E}_{c \sim q(c|y, \mathbf{a}, \mathbf{b})} \left [\nabla \log p(y|\mathbf{a},\mathbf{b}; \theta_c) + \log p(y|\mathbf{a},\mathbf{b}; \theta_c)  \nabla \log q(c|y, \mathbf{a}, \mathbf{b})\right]
$$


At inference time, to get $p(y|\mathbf{a}, \mathbf{b}; \theta)$ we use enumeration to calculate it exactly. For posterior inference, we can either use $q(c| y, \mathbf{a}, \mathbf{b})$ to approximate the true posterior or use Bayes rule to calculate the posterior exactly.

To interpret what specialized knowledge each component $c$ learns, we can find those examples whose posterior reaches maximum at $c$. 

When a model is trained, use the following test function to produce predictions, and then upload your best result to the kaggle competition:  https://www.kaggle.com/c/harvard-cs287-s19-hw4

In [ ]:
def test_code(model):
    "All models should be able to be run with following command."
    upload = []
    # Update: for kaggle the bucket iterator needs to have batch_size 10
    test_iter = torchtext.data.BucketIterator(test, train=False, batch_size=10)
    for batch in test_iter:
        # Your prediction data here (don't cheat!)
        probs = model(batch.text)
        # here we assume that the name for dimension classes is `classes`
        _, argmax = probs.max('classes')
        upload += argmax.tolist()

    with open("predictions.txt", "w") as f:
        for u in upload:
            f.write(str(u) + "\n")

In addition, you should put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/nlp-template